In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import networkx as nx
import numpy as np
from scipy.sparse.linalg import eigs
import scipy.sparse as sp

from randcolorgraphs.utils.calculate_katz import calculate_katz

def get_edge_vector_from_adj_matrix(adj_matrix):
    sparse_matrix = sp.csr_matrix(adj_matrix)
    row_indices, col_indices = sparse_matrix.nonzero()
    edge_vector = np.vstack((row_indices, col_indices)).T
    return edge_vector

def generate_adjacency_matrix_and_katz_vector(n, p, seed=42, alpha=0.1, beta=1.0):
    G = nx.erdos_renyi_graph(n, p, directed=True, seed=seed)
    A_G = nx.adjacency_matrix(G).astype(np.float64)


    eigenvalues, _ = eigs(A_G, k=1, which='LM')  # 'LM': Largest Magnitude, tol is tolerance
    spectral_radius_G = np.abs(eigenvalues).max()
    print(f"Spectral Radius: {spectral_radius_G}, alpha_max = {1/spectral_radius_G}")
    print("calculating katz")
    katz_centrality = calculate_katz(A_G, alpha = alpha, beta=beta)
    print("done calculating katz")
    # Reorder nodes based on Katz centrality
    sorted_indices = np.argsort(katz_centrality)
    sorted_nodes = np.array(G.nodes())[sorted_indices]

    # Generate the adjacency matrix
    adj_matrix = nx.adjacency_matrix(G, nodelist=sorted_nodes)

    sorted_katz_vector = katz_centrality[sorted_indices]

    assert np.allclose(sorted_katz_vector, calculate_katz(adj_matrix.astype(np.float64), alpha = alpha, beta=beta))
    return adj_matrix, sorted_katz_vector


In [ ]:
n = 500
p = 6/n
adj_matrix, katz_vector = generate_adjacency_matrix_and_katz_vector(n=n, p=p, alpha=0.1)
edges = get_edge_vector_from_adj_matrix(adj_matrix)

In [ ]:
from randcolorgraphs.algorithms.linear_scalarization.unoptimized_greedy_search import unoptimized_greedy_search_linear_scalarization
w = np.exp(-9)

# Can go to n ~ 500; smaller w -> much slower
clusters, _ = unoptimized_greedy_search_linear_scalarization(katz_vector, adj_matrix.todense(), np.array([0]*len(katz_vector), dtype=np.int64), w=w, pam_cluster_dist=2)

In [ ]:
from randcolorgraphs.algorithms.linear_scalarization.optimal_contiguous.optimal_contiguous_linear_scalarization_algo import optimal_contiguous_linear_scalarization_algo

# Can go up to n ~ 2_000; indepedented of w
clusters = optimal_contiguous_linear_scalarization_algo(katz_vector, adj_matrix.todense(), w)